# OOF are not being saved correctly

In [130]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit, KFold, GroupKFold
from sklearn.metrics import mean_absolute_error

In [63]:
oof_m021 = pd.read_csv('../oof/M021_0626_0129_oof_lgb_2folds_1.5972CV_500000iter_0.3lr.csv')

In [5]:
train = pd.read_csv('../input/train.csv')

In [58]:
X = train[['atom_index_0','atom_index_1','type']]
mol_group = train[['molecule_name','type']]
y = train['scalar_coupling_constant']
oof_df = train[['id', 'type', 'scalar_coupling_constant']].copy()
oof_df['oof_preds'] = 0

In [133]:
N_FOLDS = 2
RANDOM_STATE = 529
folds = GroupKFold(n_splits=N_FOLDS)
#folds = KFold(n_splits=N_FOLDS, random_state=RANDOM_STATE)
oof = np.zeros(len(X_type))
for bond_type in train['type'].unique():
    X_type = X.loc[X['type'] == bond_type]
    y_type = y.iloc[X_type.index]
    oof = np.zeros(len(X_type))
    mol_group_type = mol_group.loc[mol_group['type'] == bond_type]['molecule_name']
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X_type, groups=mol_group_type)):
        print(len(train_idx), len(valid_idx))
    #for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X_type)):
        X_train, X_valid = X_type.iloc[train_idx], X_type.iloc[valid_idx]
        y_train, y_valid = y_type.iloc[train_idx], y_type.iloc[valid_idx]
        y_pred_valid = y_valid + np.random.random_sample(len(y_valid)) # Fake predictions
        oof[valid_idx] = y_pred_valid.reset_index(drop=True).values
        print('Bond {} - Fold {}'.format(bond_type, fold_n+1))
        print(mean_absolute_error(y_valid, y_pred_valid))
    oof_df.loc[oof_df['type'] == bond_type, 'oof_preds'] = oof
print('Overall score {:0.4f}'.format(mean_absolute_error(oof_df['scalar_coupling_constant'], oof_df['oof_preds'])))
for i, d in oof_df.groupby('type'):
    print('Type {} - score {:0.4f}'.format(i,
                                          mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds'])))

354708 354708
Bond 1JHC - Fold 1
0.49983938446579956
354708 354708
Bond 1JHC - Fold 2
0.49952961001790097
189018 189018
Bond 2JHH - Fold 1
0.49882612359538364
189018 189018
Bond 2JHH - Fold 2
0.5008167108431367
21681 21682
Bond 1JHN - Fold 1
0.5052127029312876
21682 21681
Bond 1JHN - Fold 2
0.5015677525311337
59626 59627
Bond 2JHN - Fold 1
0.5037628951894645
59627 59626
Bond 2JHN - Fold 2
0.4987568572414204
570337 570337
Bond 2JHC - Fold 1
0.5000389426007922
570337 570337
Bond 2JHC - Fold 2
0.5002245493035572
295305 295306
Bond 3JHH - Fold 1
0.4992094774836316
295306 295305
Bond 3JHH - Fold 2
0.4993501430917144
755189 755190
Bond 3JHC - Fold 1
0.5003828299184663
755190 755189
Bond 3JHC - Fold 2
0.4999987527317816
83207 83208
Bond 3JHN - Fold 1
0.500555791554801
83208 83207
Bond 3JHN - Fold 2
0.49961672331823404
Overall score 0.5000
Type 1JHC - score 0.4997
Type 1JHN - score 0.5034
Type 2JHC - score 0.5001
Type 2JHH - score 0.4998
Type 2JHN - score 0.5013
Type 3JHC - score 0.5002
Type 3

In [128]:
len(train_idx)

567105

In [125]:
len(valid_idx)

142311

In [102]:
oof_df.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,84.860442
1,1,2JHH,-11.2570,0.000000
2,2,2JHH,-11.2548,0.000000
3,3,2JHH,-11.2543,0.000000
4,4,1JHC,84.8074,85.382138


11.939062796822023

In [91]:
oof_df.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,84.969391
1,1,2JHH,-11.2570,0.000000
2,2,2JHH,-11.2548,0.000000
3,3,2JHH,-11.2543,0.000000
4,4,1JHC,84.8074,85.742468


In [92]:
oof_df.loc[oof_df['type'] == '2JHH'].head()

,id,type,scalar_coupling_constant,oof_preds
1,1,2JHH,-11.2570,0.0
2,2,2JHH,-11.2548,0.0
3,3,2JHH,-11.2543,0.0
5,5,2JHH,-11.2541,0.0
6,6,2JHH,-11.2548,0.0


In [80]:
X_type.head()

,atom_index_0,atom_index_1,type
73,3,2,3JHN
78,4,2,3JHN
82,5,2,3JHN
213,4,2,3JHN
218,5,2,3JHN


In [78]:
X_type.head()

,atom_index_0,atom_index_1,type
73,3,2,3JHN
78,4,2,3JHN
82,5,2,3JHN
213,4,2,3JHN
218,5,2,3JHN


In [77]:
train['type'].unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [64]:
oof_df.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,84.969391
1,1,2JHH,-11.2570,0.000000
2,2,2JHH,-11.2548,0.000000
3,3,2JHH,-11.2543,0.000000
4,4,1JHC,84.8074,85.742468


In [82]:
oof_df.head()

,id,type,scalar_coupling_constant,oof_preds
0,0,1JHC,84.8076,84.969391
1,1,2JHH,-11.2570,0.000000
2,2,2JHH,-11.2548,0.000000
3,3,2JHH,-11.2543,0.000000
4,4,1JHC,84.8074,85.742468
